In [ ]:
import pandas as pd
from io import BytesIO
import requests
from collections import Counter

def fetch_data(url, skiprows=0):
    r = requests.get(url)
    data = r.content

    return pd.read_csv(BytesIO(data), index_col=0, skiprows=skiprows)

practice_sets = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=1907623426&format=csv", 0)
practice_sets

In [ ]:
categories = scrubbed_sets.keys()[3:]
repl_nan = { i : False for i in scrubbed_sets.keys()[3:]}
repl_checked = { i : 'x' for i in scrubbed_sets.keys()[3:]}

scrubbed_sets = practice_sets.fillna(value=repl_nan).replace(repl_checked, True)
scrubbed_sets.head()

In [ ]:
gb = scrubbed_sets.groupby('Set Id')
cat_avgs = (gb[categories].sum() / gb[categories].count() * 20).mean()
cat_avgs[categories]

This will calculate the count of each question type for each practice set we have data for, along with overall average and variance.

Note: the final result is normalized to be a percent of each 20 question test to make it directly comparable to our catalog percentages.

In [201]:
gb2 = scrubbed_sets.groupby(['Set Id', 'Type'])[categories]
qs_in_set = [25 for i in range(0, 8)] + [20 for i in range(0, 24)] # 8 sets, 4 rows per set
qs_per_set = gb2.count()['Type'].divide(qs_in_set, axis='index') * 20
#qs_per_set.groupby('Type').mean()
#qs_per_set.groupby('Type').var()
#qs_per_set.groupby('Type').max() - qs_per_set.groupby('Type').min()
target_weights = qs_per_set.groupby('Type').mean() / 20
target_weights

Type
MA    0.0875
MC    0.4400
NE    0.0950
QC    0.3775
dtype: float64